In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


#Reading all the file
yello_taxi_2023_01 = pd.read_parquet("Data/2023_01.parquet")
yello_taxi_2023_02 = pd.read_parquet("Data/2023_02.parquet")
yello_taxi_2023_03 = pd.read_parquet("Data/2023_03.parquet")
yello_taxi_2023_04 = pd.read_parquet("Data/2023_04.parquet")
yello_taxi_2023_05 = pd.read_parquet("Data/2023_05.parquet")
yello_taxi_2023_06 = pd.read_parquet("Data/2023_06.parquet")
yello_taxi_2023_07 = pd.read_parquet("Data/2023_07.parquet")
yello_taxi_2023_08 = pd.read_parquet("Data/2023_08.parquet")
yello_taxi_2023_09 = pd.read_parquet("Data/2023_09.parquet")
yello_taxi_2023_10 = pd.read_parquet("Data/2023_10.parquet")
yello_taxi_2023_11 = pd.read_parquet("Data/2023_11.parquet")
yello_taxi_2023_12 = pd.read_parquet("Data/2023_12.parquet")

yello_taxi_2024_01 = pd.read_parquet("Data/2024_01.parquet")
yello_taxi_2024_02 = pd.read_parquet("Data/2024_02.parquet")
yello_taxi_2024_03 = pd.read_parquet("Data/2024_03.parquet")
yello_taxi_2024_04 = pd.read_parquet("Data/2024_04.parquet")
yello_taxi_2024_05 = pd.read_parquet("Data/2024_05.parquet")
yello_taxi_2024_06 = pd.read_parquet("Data/2024_06.parquet")
yello_taxi_2024_07 = pd.read_parquet("Data/2024_07.parquet")
yello_taxi_2024_08 = pd.read_parquet("Data/2024_08.parquet")
yello_taxi_2024_09 = pd.read_parquet("Data/2024_09.parquet")
yello_taxi_2024_10 = pd.read_parquet("Data/2024_10.parquet")
yello_taxi_2024_11 = pd.read_parquet("Data/2024_11.parquet")


sum_dataset = pd.concat([yello_taxi_2023_01,yello_taxi_2023_02,yello_taxi_2023_03,
            yello_taxi_2023_04,yello_taxi_2023_05,yello_taxi_2023_06,
            yello_taxi_2023_07,yello_taxi_2023_08,yello_taxi_2023_09, 
            yello_taxi_2023_10, yello_taxi_2023_11, yello_taxi_2023_12,
            yello_taxi_2024_01, yello_taxi_2024_02, yello_taxi_2024_03,
            yello_taxi_2024_04, yello_taxi_2024_05, yello_taxi_2024_06,
            yello_taxi_2024_07, yello_taxi_2024_08, yello_taxi_2024_09, 
            yello_taxi_2024_10, yello_taxi_2024_11
            ])


数据预处理：

In [2]:
def safe_preprocess(df):
    # 保留原始数据副本
    raw_df = df
    
    # 逐步过滤（添加调试输出）
    print("\n[1] 初始数据量:", len(raw_df))
    
    # 第一轮过滤：基础有效性过滤
    cond = (
        raw_df["passenger_count"].between(1, 6) &
        raw_df["trip_distance"].between(0.1, 100) &
        (raw_df["total_amount"] > 0) &
        (raw_df["tip_amount"] >= 0) &
        raw_df["tpep_pickup_datetime"].notna() &
        raw_df["tpep_dropoff_datetime"].notna()
    )
    stage1 = raw_df[cond]
    print("[2] 基础过滤后:", len(stage1))
    
    # 时间逻辑过滤
    time_cond = (stage1["tpep_dropoff_datetime"] > stage1["tpep_pickup_datetime"])
    stage1 = stage1[time_cond]
    print("[3] 时间过滤后:", len(stage1))
    
    # 计算时间相关特征
    stage1["trip_duration"] = (stage1["tpep_dropoff_datetime"] - stage1["tpep_pickup_datetime"]).dt.total_seconds() / 3600
    stage1["speed_kmh"] = stage1["trip_distance"] / stage1["trip_duration"]
    
    # 速度过滤（放宽限制）
    speed_cond = stage1["speed_kmh"].between(0.5, 150)  # 包含低速和高速公路速度
    stage1 = stage1[speed_cond]
    print("[4] 速度过滤后:", len(stage1))
    
    # 小费比例计算与过滤
    stage1["tip_ratio"] = stage1["tip_amount"] / stage1["total_amount"]
    ratio_cond = stage1["tip_ratio"].between(0, 1)  # 允许100%小费
    final_df = stage1[ratio_cond]
    print("[5] 最终数据量:", len(final_df))
    
    # 时间特征（确保列存在）
    final_df["dropoff_hour"] = final_df["tpep_dropoff_datetime"].dt.hour
    # 生成时间周期特征（正弦+余弦编码）
    final_df['time_sin'] = np.sin(2 * np.pi * final_df["dropoff_hour"] / 24)
    final_df['time_cos'] = np.cos(2 * np.pi * final_df["dropoff_hour"] / 24)
    final_df["time_angle"] = np.arctan2(final_df["time_sin"], final_df["time_cos"])
    # 生成其他时间相关特征
    final_df['is_early_morning'] = final_df["dropoff_hour"].between(4, 6).astype(int)
    final_df['is_rush_hour'] = final_df["dropoff_hour"].isin([8, 17, 18,21,22]).astype(int)
    
    final_df["dropoff_dayofweek"] = final_df["tpep_dropoff_datetime"].dt.dayofweek
    final_df["is_weekend"] = final_df["dropoff_dayofweek"].isin([5, 6]).astype(int)
    
    return final_df

# 正确调用方式：从原始数据开始处理
positive_pay_sum_dataset = safe_preprocess(sum_dataset)


[1] 初始数据量: 75811575
[2] 基础过滤后: 67854154
[3] 时间过滤后: 67851501
[4] 速度过滤后: 67781859
[5] 最终数据量: 67781856


/var/folders/ms/kdx178sn0hn6k06q5wb3d6km0000gn/T/ipykernel_33721/1072556880.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["dropoff_hour"] = final_df["tpep_dropoff_datetime"].dt.hour
/var/folders/ms/kdx178sn0hn6k06q5wb3d6km0000gn/T/ipykernel_33721/1072556880.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['time_sin'] = np.sin(2 * np.pi * final_df["dropoff_hour"] / 24)
/var/folders/ms/kdx178sn0hn6k06q5wb3d6km0000gn/T/ipykernel_33721/1072556880.py:44: SettingWithCopyWarning: 
A va

In [12]:
print(positive_pay_sum_dataset["speed_kmh"])

0           6.901186
1          10.448549
2          11.811765
4           7.920000
5           8.975610
             ...    
3272690     8.648649
3272691     8.221860
3272692     7.855787
3272693     9.214854
3272694     8.411683
Name: speed_kmh, Length: 67781856, dtype: float64


列出考虑因素和特征，以及裁剪数据

In [3]:
# 基础特征列表（不包含需要编码的类别特征）
base_features = [
    "passenger_count",
    "trip_distance",
    "speed_kmh",
    "time_sin",
    'time_cos',
    'is_early_morning',
    'is_rush_hour',
    "time_angle",
    "is_weekend"       # 已经是二值特征
]

# 原始数据获取（包含需要编码特征）
processed_df = positive_pay_sum_dataset[base_features + ["tip_ratio"]].copy()


# 安全抽样（先抽样再划分数据集）
sample_df = processed_df.sample(frac=0.02)

# 划分数据集（必须在此步骤之后进行标准化）
X = sample_df[base_features]
y = sample_df["tip_ratio"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

# 创建数据副本避免修改原始数据
X_train_processed = X_train
X_test_processed = X_test


print(X_train_processed.count())

passenger_count     1084509
trip_distance       1084509
speed_kmh           1084509
time_sin            1084509
time_cos            1084509
is_early_morning    1084509
is_rush_hour        1084509
time_angle          1084509
is_weekend          1084509
dtype: int64


模型评估函数：

In [4]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    return {
        "MAE": mean_absolute_error(y_test, y_pred),
        "MSE": mean_squared_error(y_test, y_pred),
        "R2": r2_score(y_test, y_pred)
    }

线性回归模型训练

In [5]:
# 线性回归
lr_model = LinearRegression()
lr_model.fit(X_train_processed, y_train)

#评估模型
lr_metrics = evaluate_model(lr_model, X_test_processed, y_test)
print(f"MAE: {lr_metrics['MAE']:.4f}")
print(f"MSE: {lr_metrics['MSE']:.4f}")
print(f"R²: {lr_metrics['R2']:.4f}")

MAE: 0.0632
MSE: 0.0055
R²: 0.0056


LightBGM

In [8]:
import lightgbm as lgb
LBGM_model = lgb.LGBMRegressor(
    objective='regression',
    n_estimators=2000,
    learning_rate=0.02,
    max_depth=6,
    num_leaves=31,
    subsample=0.7,
    reg_alpha=0.1,              # L1正则化
    reg_lambda=0.1,             # L2正则化
    random_state=42
)
LBGM_model.fit(X_train, y_train, eval_set=[(X_test, y_test)])
#评估模型
LGBM_metrics = evaluate_model(LBGM_model, X_test_processed, y_test)
print(f"MAE: {LGBM_metrics['MAE']:.4f}")
print(f"MSE: {LGBM_metrics['MSE']:.4f}")
print(f"R²: {LGBM_metrics['R2']:.4f}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010116 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 587
[LightGBM] [Info] Number of data points in the train set: 1084509, number of used features: 9
[LightGBM] [Info] Start training from score 0.121620
MAE: 0.0625
MSE: 0.0054
R²: 0.0191


Kmean

In [13]:
knn_model = KNeighborsRegressor(n_neighbors=5)
knn_model.fit(X_train_processed, y_train)

#评估模型
knn_metrics = evaluate_model(knn_model, X_test_processed, y_test)
print(f"MAE: {knn_metrics['MAE']:.4f}")
print(f"MSE: {knn_metrics['MSE']:.4f}")
print(f"R²: {knn_metrics['R2']:.4f}")

MAE: 0.0656
MSE: 0.0065
R²: -0.1736


Random forest

In [11]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_processed, y_train)

#评估模型
knn_metrics = evaluate_model(rf_model, X_test_processed, y_test)
print(f"MAE: {knn_metrics['MAE']:.4f}")
print(f"MSE: {knn_metrics['MSE']:.4f}")
print(f"R²: {knn_metrics['R2']:.4f}")


MAE: 0.0648
MSE: 0.0062
R²: -0.1122


SVM

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, PowerTransformer
from sklearn.metrics import mean_absolute_error, r2_score

# ====================
# 设备配置
# ====================
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# ====================
# 高级数据预处理
# ====================
# 假设原始数据包含以下特征：
# ['trip_distance', 'speed_kmh', 'dropoff_hour', 'period_Morning_Rush', ...]

# 定义特征处理管道
numeric_features = ['trip_distance', 'speed_kmh', 'dropoff_hour']
categorical_features = [col for col in X_train.columns if 'period_' in col]

preprocessor = ColumnTransformer([
    # 数值特征处理管道
    ('numeric', 
     Pipeline([
        ('scaler', RobustScaler()),
        ('power', PowerTransformer(method='yeo-johnson'))
     ]),  # 注意这里不需要指定列名
     numeric_features),  # 列名在此处指定
    
    # 分类特征直通
    ('categorical', 
     'passthrough', 
     categorical_features)
])

# 应用预处理
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# 添加交互特征
X_train_processed = pd.DataFrame(X_train_processed, 
                                columns=numeric_features + categorical_features)
X_train_processed['speed_hour'] = X_train_processed['speed_kmh'] * X_train_processed['dropoff_hour']

X_test_processed = pd.DataFrame(X_test_processed,
                               columns=numeric_features + categorical_features)
X_test_processed['speed_hour'] = X_test_processed['speed_kmh'] * X_test_processed['dropoff_hour']

# 转换目标变量（可选）
y_train_trans = np.log(y_train / (1 - y_train + 1e-8))  # logit变换
y_test_trans = np.log(y_test / (1 - y_test + 1e-8))

# ====================
# 增强模型架构
# ====================
class EnhancedSVR(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, C=1.0, epsilon=0.1):
        super().__init__()
        self.hidden = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        self.output = nn.Linear(hidden_dim, 1)
        
        # 初始化参数
        self._initialize_weights()
        
        self.C = C
        self.epsilon = epsilon

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.hidden(x)
        return self.output(x)

    def custom_loss(self, pred, target):
        errors = target - pred
        quadratic = torch.where(
            torch.abs(errors) < self.epsilon,
            0.5 * (errors**2),
            self.epsilon * torch.abs(errors) - 0.5 * (self.epsilon**2)
        )
        # 所有参数的L2正则化
        l2_reg = sum(torch.sum(param**2) for param in self.parameters()) / (2 * self.C)
        return torch.mean(quadratic) + l2_reg

# ====================
# 数据转换
# ====================
input_dim = X_train_processed.shape[1]

X_train_tensor = torch.tensor(X_train_processed.values, 
                             dtype=torch.float32, device=device)
y_train_tensor = torch.tensor(y_train_trans.values.reshape(-1, 1),
                             dtype=torch.float32, device=device)
X_test_tensor = torch.tensor(X_test_processed.values,
                            dtype=torch.float32, device=device)

# ====================
# 模型初始化
# ====================
model = EnhancedSVR(input_dim, hidden_dim=128, C=0.1, epsilon=0.05).to(device)

# ====================
# 优化配置
# ====================
optimizer = optim.AdamW([
    {'params': model.hidden.parameters(), 'lr': 1e-3},
    {'params': model.output.parameters(), 'lr': 1e-2}
], weight_decay=0.01)

# 方案1：使用 CyclicLR（适合batch级别调度）
scheduler = optim.lr_scheduler.CyclicLR(
    optimizer,
    base_lr=1e-4,
    max_lr=1e-2,
    step_size_up=500,       # 每个周期上升步数
    step_size_down=500,     # 每个周期下降步数
    cycle_momentum=False,
    mode='triangular2'      # 更稳定的学习率变化
)

# ====================
# 改进训练循环
# ====================
best_loss = float('inf')
patience_counter = 0
patience = 50
min_delta = 0.001
history = {'train_loss': [], 'val_mae': [], 'val_r2': []}

from torch.utils.data import DataLoader, TensorDataset

# 创建DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=4096, shuffle=True)

best_loss = float('inf')
patience_counter = 0
history = {'train_loss': [], 'val_mae': [], 'val_r2': []}

for epoch in range(3000):
    model.train()
    epoch_loss = 0.0
    
    # Batch训练
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        
        preds = model(X_batch)
        loss = model.custom_loss(preds, y_batch)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()  # 每个batch更新学习率
        
        epoch_loss += loss.item()
    
    # 计算平均epoch loss
    avg_epoch_loss = epoch_loss / len(train_loader)
    
    # 验证与早停
    model.eval()
    with torch.no_grad():
        test_pred = model(X_test_tensor).cpu().numpy()
        test_mae = mean_absolute_error(y_test_trans, test_pred)
        test_r2 = r2_score(y_test_trans, test_pred)
        
        if avg_epoch_loss < (best_loss - min_delta):
            best_loss = avg_epoch_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1
    
    # 打印进度
    if epoch % 10 == 0:
        current_lr = optimizer.param_groups[0]['lr']
        print(f"Epoch {epoch:04d} | Loss: {avg_epoch_loss:.4f} | "
              f"LR: {current_lr:.2e} | "
              f"Test MAE: {test_mae:.4f} | "
              f"Test R²: {test_r2:.4f}")
    
    if patience_counter >= patience:
        print(f"\nEarly stopping triggered at epoch {epoch}")
        break




# ====================
# 最终评估与结果转换
# ====================
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

with torch.no_grad():
    # 转换预测结果回原始比例
    test_pred = torch.sigmoid(torch.tensor(test_pred)).numpy()
    
    # 计算最终指标
    final_mae = mean_absolute_error(y_test, test_pred)
    final_r2 = r2_score(y_test, test_pred)
    
    print("\n=== 最终测试性能 ===")
    print(f"MAE: {final_mae:.4f}")
    print(f"R²: {final_r2:.4f}")

# ====================
# 特征重要性分析（改进版）
# ====================
# 获取中间层激活值
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = input[0].detach()
    return hook

model.hidden[0].register_forward_hook(get_activation('hidden'))

with torch.no_grad():
    _ = model(X_test_tensor[:1000])
    hidden_activation = activation['hidden'].cpu().numpy()

# 计算特征重要性
feature_importance = np.abs(hidden_activation).mean(axis=0)
sorted_idx = np.argsort(feature_importance)[::-1]

print("\n隐藏层特征重要性（Top 10）：")
for idx in sorted_idx[:10]:
    print(f"Feature {idx}: {feature_importance[idx]:.4f}")

# 可视化
plt.figure(figsize=(10, 6))
plt.bar(range(len(feature_importance)), feature_importance)
plt.title("Hidden Layer Feature Importance")
plt.xlabel("Feature Index")
plt.ylabel("Mean Absolute Activation")
plt.show()